In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import LabelPowerset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_drug.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/test_features.csv


In [2]:
train_feature = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col=0,header=0)
train_drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv',index_col=0,header=0)
train_target = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col=0,header=0)
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv',index_col=0,header=0)
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col=0,header=0)
train_feature.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [3]:
train_feature['cp_type'] = train_feature['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})

train_feature['cp_dose'] = train_feature['cp_dose'].map({'D1':0,'D2':1})

train_feature.head()

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,0,24,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,0,72,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,0,48,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,0,48,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,0,72,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
traindataloader = torch.utils.data.DataLoader(torch.from_numpy(train_feature.to_numpy().astype(np.float32)).to(device),
                                              shuffle=True, 
                                              batch_size=train_feature.shape[0])

In [5]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        #Encoder
        self.encoder = nn.Sequential(
            nn.Linear(in_features=875, out_features=4096),
            nn.ReLU(True), 
            nn.Linear(in_features=4096, out_features=2048), 
            nn.ReLU(True), 
            nn.Linear(in_features=2048, out_features=1024),
            nn.ReLU(True),
            nn.Linear(in_features=1024, out_features=512),
            nn.ReLU(True),
            nn.Linear(in_features=512, out_features=256),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(in_features=256, out_features=512),
            nn.ReLU(True), 
            nn.Linear(in_features=512, out_features=1024), 
            nn.ReLU(True), 
            nn.Linear(in_features=1024, out_features=2048),
            nn.ReLU(True),
            nn.Linear(in_features=2048, out_features=4096),
            nn.ReLU(True),
            nn.Linear(in_features=4096, out_features=875),
            nn.ReLU(True))
        
    def forward(self, x):
        x = self.encoder(x)
#         x.register_hook(lambda grad: print(x))
        x = self.decoder(x)
        return x

In [6]:
Lr_Rate = 1e-4
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=Lr_Rate)
model = model.to(device)
print(model)


Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=875, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=4096, out_features=2048, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=512, out_features=256, bias=True)
    (9): ReLU(inplace=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=1024, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=1024, out_features=2048, bias=True)
    (5): ReLU(inplace=True)
    (6): Linear(in_features=2048, out_features=4096, bias=True)
    (7): ReLU(inplace=True)
    (8): Linear(in_features=4096, out_features=875, bias=True)
    (9): ReLU(inplace=True)
  )
)


In [ ]:
for epochs in range(1000):
    train_loss = 0
    epoch_loss = 0
    for trdata in traindataloader:
        optimizer.zero_grad()
        out = model(trdata)
        train_loss = criterion(out,trdata)
        epoch_loss += train_loss
        train_loss.backward()
        optimizer.step()
        
    if epochs % 100 == 0:
        print("Epoch = ", epochs, "||", "epoch_loss = ", epoch_loss.detach().cpu().numpy().tolist())

Epoch =  0 || epoch_loss =  4.850584506988525
Epoch =  100 || epoch_loss =  1.7853009700775146
Epoch =  200 || epoch_loss =  1.7844277620315552
Epoch =  300 || epoch_loss =  1.7843141555786133
Epoch =  400 || epoch_loss =  1.7827601432800293


In [ ]:
p1 = model.encoder(torch.from_numpy(train_feature.to_numpy().astype(np.float32)).to(device))
p1 = p1.cpu().detach().numpy()
X_train, X_test, y_train, y_test = train_test_split(p1, train_target, test_size=0.20, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
name = 'Random_Forest'
output = {}
output = {}
clf = RandomForestClassifier()
classifier = LabelPowerset(clf)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)
accuracyscore = accuracy_score(y_test,predictions)
output.update({name:accuracyscore})

In [ ]:
test_features.head()

In [ ]:
test_features['cp_type'] = test_features['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
test_features['cp_dose'] = test_features['cp_dose'].map({'D1':0,'D2':1})
sample_sub = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv',index_col=0,header=0)
p1 = model.encoder(torch.from_numpy(test_features.to_numpy().astype(np.float32)).to(device))
p1 = p1.cpu().detach().numpy()
predictions = classifier.predict(p1)
predictions = pd.DataFrame(predictions.toarray())
predictions.index = sample_sub.index
predictions.columns = sample_sub.columns
predictions.to_csv('submission.csv', index=False)